In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_MA_journal
@pyimport GLS_Apr_weekday_PM_journal

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in transpose(::Ptr{PyCall.PyObject_struct}) at ./deprecated.jl:770
 in transpose_f!(::Base.#transpose, ::Array{Ptr{PyCall.PyObject_struct},2}, ::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:369
 in transpose(::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:407
 in copy(::PyCall.PyArray{Ptr{PyCall.PyObject_struct},2}) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:337
 in convert(::Type{Array{Ptr{PyCall.PyObject_struct},N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:453
 in convert(::Type{Array{PyCall.PyObject,N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:484
 in convert(::Type{PyCall.PyAny}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/conversions.jl:806
 in #55 at ./<missing>:0 [inlined]
 in next at ./generator.jl:26 [inlined]
 in collect_to!(::Array{Expr,1}, ::Base.Generator{Array{Tuple{AbstractString,PyCall.PyObject},1},PyCall.##55#59}, ::Int64, ::Int64) at

In [2]:
numNodes = Compute_Jacobian_MA_journal.numNodes;
numLinks = Compute_Jacobian_MA_journal.numLinks;
numODpairs = Compute_Jacobian_MA_journal.numODpairs;
numZones = Compute_Jacobian_MA_journal.numZones
od_pairs = Compute_Jacobian_MA_journal.od_pairs;
link_list_js = Compute_Jacobian_MA_journal.link_list_js;
link_length_list = Compute_Jacobian_MA_journal.link_length_list;

flow_observ = GLS_Apr_weekday_PM_journal.x_

258×30 Array{Float64,2}:
 1393.18     806.378    762.959        …  1122.05    1513.32     356.411 
 1245.77    1047.31    1023.88            1058.72    1286.13     737.719 
 2175.98    1580.96    1289.54            1310.68    1660.56    1011.89  
 2206.51    1358.68    1100.85            1375.95    2022.21     856.967 
   27.0447   279.236    279.358            114.702    182.177    319.842 
  143.919    260.583    207.122        …   112.767     47.7074    93.4537
  128.953     91.0833    77.1364           105.58     127.956     72.177 
  128.953     91.0833    77.1364           105.58     127.956     72.177 
 1381.89    1661.69    1435.11            1024.77    1282.9     1395.51  
 1061.96    1922.71    1849.59             893.572    782.416   2032.03  
   85.5516   284.675    357.062        …    50.566     29.8585   369.656 
  258.077    264.587    203.507            118.433    303.162    114.442 
  322.493    399.126    250.17             333.773    473.913    174.446 
    ⋮        

In [3]:
include("../Julia_files/initia_data.jl");
include("prepare_data_MA_journal.jl");
include("extract_data_MA_journal.jl");
include("../Julia_files/tap_MSA.jl");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/prepare_data_MA_journal.jl:183
 in include_from_node1(::String) at ./loading.jl:488
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/extract_data_MA_journal.jl, in expression starting on line 10
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_de

In [4]:
deg = 8
c = 0.5
lam = 10000.0

coeffs_dict_Apr_PM_ = readstring("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

9-element Array{Any,1}:
  1.0       
 -0.00303133
  0.0577207 
 -0.195677  
  0.620789  
 -0.905919  
  0.935921  
 -0.469131  
  0.108528  

In [5]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
        free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
        free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + 
        free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + 
        free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [6]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30];
# week_day_Apr_list = [6];

In [7]:
PoA_dict = Dict();
tapSocialFlowDicDict = Dict();
tapSocialFlowVecDict = Dict();

In [8]:
# demandsDict = readstring("../temp_files/demands_MA_journal.json");
# demandsDict = JSON.parse(demandsDict)

# demandsDict_ = Dict()
# for key in keys(demandsDict)
#     key_ = (parse(Int, split(split(key, ',')[1], '(')[2]), parse(Int, split(split(key, ',')[2], ')')[1]))
#     demandsDict_[key_] = demandsDict[key]
# end

# AA, BB = tapMSASocial(demandsDict_, fcoeffs);

In [9]:
for day in week_day_Apr_list

    demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/results/demandsDictFixed$(day)_journal.json");
    demandsDict = JSON.parse(demandsDict)

    demandsDict_ = Dict()
    for key in keys(demandsDict)
        key_ = (parse(Int, split(split(key, ',')[1], '(')[2]), parse(Int, split(split(key, ',')[2], ')')[1]))
        demandsDict_[key_] = demandsDict[key]
    end

#     demandsDict_

#     tapFlowDicDict = Dict()
#     tapFlowVecDict = Dict()
#     tapFlowDicDict[day], tapFlowVecDict[day] = tapMSA(demandsDict_, fcoeffs);

#     tapFlowVecDict[day]

    tapSocialFlowDicDict[day], tapSocialFlowVecDict[day] = tapMSASocial(demandsDict_, fcoeffs);

#     tapSocialFlowVecDict[day]

#     flow_observ[:, day]

    # PoA_dict[day] = socialObj(tapFlowVecDict[day]) / socialObj(tapSocialFlowVecDict[day])

    PoA_dict[day] = socialObj(flow_observ[:, day]) / socialObj(tapSocialFlowVecDict[day]);
end

In [10]:
PoA_dict

Dict{Any,Any} with 21 entries:
  30 => 1.89434
  2  => 1.76427
  16 => 1.67644
  11 => 1.3032
  26 => 1.34687
  9  => 1.23792
  25 => 1.44386
  10 => 1.41722
  19 => 1.73921
  17 => 1.21251
  6  => 1.21549
  24 => 1.66795
  4  => 1.75585
  3  => 1.88139
  5  => 1.60888
  20 => 1.34646
  23 => 1.53948
  13 => 1.30214
  27 => 1.5314
  12 => 1.62308
  18 => 1.28076

In [11]:
tapSocialFlowVecDict

Dict{Any,Any} with 21 entries:
  30 => [583.945,349.959,1282.25,812.25,0.0,0.0,1265.91,1306.98,1245.91,2319.33…
  2  => [581.116,299.312,1251.18,813.664,0.0,0.0,1226.52,1219.92,1161.93,2094.1…
  16 => [372.499,187.498,769.741,517.019,0.0,0.0,759.541,745.386,731.38,1295.16…
  11 => [563.341,268.266,1260.3,838.389,0.0,0.0,1231.83,1239.11,1152.3,2090.69 …
  26 => [583.945,349.959,1282.25,812.25,0.0,0.0,1265.91,1306.98,1245.91,2319.33…
  9  => [581.116,299.312,1251.18,813.664,0.0,0.0,1226.52,1219.92,1161.93,2094.1…
  25 => [583.945,349.959,1282.25,812.25,0.0,0.0,1265.91,1306.98,1245.91,2319.33…
  10 => [563.341,268.266,1260.3,838.389,0.0,0.0,1231.83,1239.11,1152.3,2090.69 …
  19 => [357.647,167.377,750.226,483.344,0.0,0.0,752.272,723.959,702.484,1246.7…
  17 => [587.958,303.932,1260.33,836.064,0.0,0.0,1229.03,1206.13,1168.67,2063.5…
  6  => [367.795,184.638,764.563,503.18,0.0,0.0,757.989,753.912,726.738,1314.29…
  24 => [580.363,324.078,1285.97,804.582,0.0,0.0,1277.93,1318.84,1267.58,2321.

In [12]:
outfile = open("../temp_files/PoA_dict_Apr_PM_journal.json", "w")

JSON.print(outfile, PoA_dict)

close(outfile)

In [13]:
outfile = open("../temp_files/tapSocialFlowVecDict_Apr_PM_journal.json", "w")

JSON.print(outfile, tapSocialFlowVecDict)

close(outfile)